In [2]:
import tensorflow as tf
import numpy as np
import PIL.Image as pilimg
import os
import glob

In [3]:
os.getcwd()

'C:\\Users\\kyj23\\Desktop\\my\\practice\\python'

In [4]:
pathName=os.path.join(os.getcwd(),'pcb')
pathName

'C:\\Users\\kyj23\\Desktop\\my\\practice\\python\\pcb'

In [5]:
train = []

def random_crop(image):
    cropped_image = tf.image.random_crop(image, size=[512,512,3])
    return cropped_image

i=0
for pth in glob.glob(pathName+'/*')[0:]:
    filename = pth
    image = pilimg.open(filename)
    image = np.array(image)
    cropped_image = tf.image.random_crop(image, size=[512, 512, 3])
    train.insert(i, cropped_image)
    i = i+1
    
train = np.array(train, dtype="float32")
train /= 255.0
train.shape

(16, 512, 512, 3)

In [6]:
train

array([[[[0.65882355, 0.654902  , 0.6862745 ],
         [0.6627451 , 0.65882355, 0.6901961 ],
         [0.6627451 , 0.65882355, 0.6901961 ],
         ...,
         [0.72156864, 0.7176471 , 0.7490196 ],
         [0.72156864, 0.7176471 , 0.7490196 ],
         [0.72156864, 0.7176471 , 0.7490196 ]],

        [[0.6745098 , 0.67058825, 0.7019608 ],
         [0.6745098 , 0.67058825, 0.7019608 ],
         [0.6745098 , 0.67058825, 0.7019608 ],
         ...,
         [0.72156864, 0.7176471 , 0.7490196 ],
         [0.72156864, 0.7176471 , 0.7490196 ],
         [0.72156864, 0.7176471 , 0.7490196 ]],

        [[0.69411767, 0.6901961 , 0.72156864],
         [0.69411767, 0.6901961 , 0.72156864],
         [0.69411767, 0.6901961 , 0.72156864],
         ...,
         [0.72156864, 0.7176471 , 0.7490196 ],
         [0.72156864, 0.7176471 , 0.7490196 ],
         [0.72156864, 0.7176471 , 0.7490196 ]],

        ...,

        [[0.62352943, 0.6862745 , 0.6431373 ],
         [0.47058824, 0.53333336, 0.49019608]

In [7]:
import tensorflow.keras.layers as layers
from keras.models import Model
from keras import losses

In [8]:
input_e = tf.keras.Input(shape=(512,512,3))
        
conv1 = layers.Conv2D(32, (3,3), activation='relu', padding = 'same')(input_e)
mp1 = layers.MaxPooling2D((2,2))(conv1)

conv2 = layers.Conv2D(64, (3,3), activation='relu', padding = 'same')(mp1)
mp2 = layers.MaxPooling2D((2,2))(conv2)
        
conv3 = layers.Conv2D(128, (3,3), activation='relu', padding = 'same')(mp2)
mp3 = layers.MaxPooling2D((2,2))(conv3)
        
conv4 = layers.Conv2D(256, (3,3), activation='relu', padding = 'same')(mp3)
mp4 = layers.MaxPooling2D((2,2))(conv4)
        
output_e = layers.Conv2D(512, (3,3), activation='relu', padding = 'same')(mp4)

convt1 = layers.Conv2DTranspose(256, (3,3), activation='relu', padding='same')(output_e) #512아니고 256
upsamp1 = layers.UpSampling2D((2,2))(convt1)
skipcon1 = layers.Concatenate(axis=3)([conv4, upsamp1])
conv6 = layers.Conv2D(256, (3,3), activation = 'relu', padding='same')(skipcon1)

convt2 = layers.Conv2DTranspose(128, (3,3), activation='relu', padding='same')(conv6)
upsamp2 = layers.UpSampling2D((2,2))(convt2)
skipcon2 = layers.Concatenate(axis=3)([conv3, upsamp2])
conv7 = layers.Conv2D(128, (3,3), activation = 'relu', padding='same')(skipcon2)

convt3 = layers.Conv2DTranspose(64, (3,3), activation='relu', padding='same')(conv7)
upsamp3 = layers.UpSampling2D((2,2))(convt3)
skipcon3 = layers.Concatenate(axis=3)([conv2, upsamp3])
conv8 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(skipcon3)

convt4 = layers.Conv2DTranspose(32, (3,3), activation='relu', padding='same')(conv8)
upsamp4 = layers.UpSampling2D((2,2))(convt4)
skipcon4 = layers.Concatenate(axis=3)([conv1, upsamp4])
conv9 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(skipcon4)

output_d = layers.Conv2DTranspose(3, (3,3), activation='relu', padding='same')(conv9)
        
model = Model(inputs=input_e, outputs=output_d)
model.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 256, 256, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
model.fit(train, train, validation_split=0.2, epochs=20, verbose=2)

Epoch 1/20
